In [1]:
from gurobipy import *
from math import *
import random
import itertools

In [2]:
#random_list = list((itertools.product(range(1,6),range(1,4))))
#print(random_list)

#随机生成n个坐标
n = 5
random_n_list = list(itertools.product(range(0, 50), range(0, 50)))
#print(random.sample(random_n_list, n))
q=[(0,0)]+random.sample(random_n_list, n)

In [3]:
q

[(0, 0), (25, 40), (28, 41), (34, 39), (11, 5), (11, 0)]

In [4]:
#n = 5
p_o=(0,0)
p_f=(0,0)
#q=((0,0),(0,40)  ,(25,25) ,(25,28) , (40,40) , (50,0),(25,26) , (26,25) , (24,24),(25,25),(30,25),(25,10),(20,5),(30,0) , (15,33)  , (19,0)  , (15,20)  , (30,23) , (0,20) , (43,0), (25,10)   );
V_v=90
V_c=18
a=0.4

In [5]:
# Create a new model
m = Model("MISOCP_2017_new")


Academic license - for non-commercial use only


In [6]:
# Create variables
Qx = {};
Qy = {};

w = {};

pto_x= {};
pto_y= {};

pl_x= {};
pl_y= {};


t1= {};
t2= {};

t= {};

T= {};

#for SOCP
Cx= {};
Cy= {};
C= {};

In [7]:
for i in range(1, n+1):
    Qx[i] = m.addVar(vtype = 'C',name='Qx[%s]'%i);
    Qy[i] = m.addVar(vtype = 'C',name='Qy[%s]'%i);
    
    pto_x[i] = m.addVar(vtype = 'C',name='pto_x[%s]'%i);
    pto_y[i] = m.addVar(vtype = 'C',name='pto_y[%s]'%i);
    
    pl_x[i] = m.addVar(vtype = 'C',name='pl_x[%s]'%i);
    pl_y[i] = m.addVar(vtype = 'C',name='pl_y[%s]'%i);
    
    t1[i] = m.addVar(lb=0.0,vtype = 'C',name='t1[%s]'%i);
    t2[i] = m.addVar(lb=0.0,vtype = 'C',name='t2[%s]'%i);
    
    t[i] = m.addVar(lb=0.0,vtype = 'C',name='t[%s]'%i);
    
    T[i] = m.addVar(lb=0.0,vtype = 'C',name='T[%s]'%i);
    
    

    for j in range(1, n+1):
        w[i,j] = m.addVar(vtype = 'B',name='w[%s,%s]'%(i,j));
        
    
T[n+1] = m.addVar(lb=0.0,vtype = 'C',name='T[%s]'%(n+1));
m.update()

In [8]:
# Add constraint: 
ctr=0;

#C5
ctr=ctr+1;
    
Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C5');
Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C5');
C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C5');
    
    
m.addConstr( Cx[ctr]  == p_o[0]-pto_x[1], 'C.5x.1'); 
m.addConstr( Cy[ctr]  == p_o[1]-pto_y[1], 'C.5y.1');
m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C.5.2');
m.addConstr( C[ctr]-(V_c*T[1])  == 0, 'C.5.3');


#C7
ctr=ctr+1;
    
Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C7');
Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C7');
C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C7');
    
    
m.addConstr( Cx[ctr]  == p_f[0]-pl_x[n], 'C.7x.1'); 
m.addConstr( Cy[ctr]  == p_f[1]-pl_y[n], 'C.7y.1');
m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C.7.2');
m.addConstr( C[ctr]-(V_c*T[n+1])  == 0, 'C.7.3');



for i in range(1, n+1):
    
    #C8
    m.addConstr( t1[i] + t2[i] - t[i] <= 0.0, 'C.8.[%s]'%i);
    
    
    #C2
    ctr=ctr+1;
    
    Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx[%s].C2'%i);
    Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy[%s].C2'%i);
    C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C[%s].C2'%i);
    
                       
    m.addConstr( Cx[ctr]  == Qx[i]-pto_x[i], 'C.2x.1.[%s]'%i); 
    m.addConstr( Cy[ctr]  == Qy[i]-pto_y[i], 'C.2y.1.[%s]'%i);
    m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C.2.2.[%s]'%i);
    m.addConstr( C[ctr]-(V_v*t1[i])  == 0, 'C.2.3.[%s]'%i);
    
    #C3
    ctr=ctr+1;
    
    Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx[%s].C3'%i);
    Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy[%s].C3'%i);
    C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C[%s].C3'%i);
    
    
    m.addConstr( Cx[ctr]  == Qx[i]-pl_x[i], 'C.3x.1.[%s]'%i); 
    m.addConstr( Cy[ctr]  == Qy[i]-pl_y[i], 'C.3y.1.[%s]'%i);
    m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C.3.2.[%s]'%i);
    m.addConstr( C[ctr]-(V_v*t2[i])  == 0, 'C.3.3.[%s]'%i);
    
    
    #C4
    ctr=ctr+1;
    
    Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx[%s].C4'%i);
    Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy[%s].C4'%i);
    C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C[%s].C4'%i);
    
    
    m.addConstr( Cx[ctr]  == pl_x[i]-pto_x[i], 'C.4x.1.[%s]'%i); 
    m.addConstr( Cy[ctr]  == pl_y[i]-pto_y[i], 'C.4y.1.[%s]'%i);
    m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C.4.2.[%s]'%i);
    m.addConstr( C[ctr]-(V_c*t[i])  == 0, 'C.4.3.[%s]'%i);
    
    
    
    #C9
    m.addConstr( Qx[i]  == quicksum(w[i,j]*q[j][0] for j in range(1,n+1)), 'C.9x.[%s]'%i); 
    m.addConstr( Qy[i]  == quicksum(w[i,j]*q[j][1] for j in range(1,n+1)), 'C.9y.[%s]'%i); 
    
    #C10
    m.addConstr( quicksum(w[i,j] for j in range(1,n+1))  == 1, 'C.10.[%s]'%i);
    
    
    #C14
    m.addConstr( t[i]  <= a, 'C.14b.[%s]'%i);
    

    #C6
    if i>1:
        ctr=ctr+1;
        
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx[%s].C6'%i);
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy[%s].C6'%i);
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C[%s].C6'%i);
    
    
        m.addConstr( Cx[ctr]  == pl_x[i-1]-pto_x[i], 'C.6x.1.[%s]'%i); 
        m.addConstr( Cy[ctr]  == pl_y[i-1]-pto_y[i], 'C.6y.1.[%s]'%i);
        m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C.6.2.[%s]'%i);
        m.addConstr( C[ctr]-(V_c*T[i])  == 0, 'C.6.3.[%s]'%i);
        
        
        
        
    
    
                       
#C11                       
for j in range(1, n+1):
    
    m.addConstr( quicksum(w[i,j] for i in range(1,n+1))  == 1, 'C.11.[%s]'%j);






In [9]:
# Set objective:


obj = quicksum(t[i] for i in range(1,n+1)) +  quicksum(T[i] for i in range(1,n+2))

m.setObjective(obj,GRB.MINIMIZE)

In [10]:
m.optimize()

Optimize a model with 93 rows, 139 columns and 289 nonzeros
Model has 21 quadratic constraints
Variable types: 114 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-01, 1e+00]
Presolve removed 35 rows and 30 columns
Presolve time: 0.01s
Presolved: 58 rows, 109 columns, 224 nonzeros
Variable types: 79 continuous, 30 integer (25 binary)

Root relaxation: objective 0.000000e+00, 24 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    4          -    0.00000      -     -    0s
     0     0    0.00000    0    8          -    0.00000      -     -    0s
     0     0    0.00000    0    8          -    0.00000      -     -    0s
H    0     0                       5.3249618    

In [11]:
for i in range(1,n+1):
    print("Q", i," : (",Qx[i].X,", ", Qy[i].X,")")

    
    print("takeoff pt", i," : (",pto_x[i].X,", ", pto_y[i].X,")")
    print("landing pt", i," : (",pl_x[i].X,", ", pl_y[i].X,")\n")

Q 1  : ( 11.0 ,  0.0 )
takeoff pt 1  : ( 4.231666144280172 ,  5.731214118730556 )
landing pt 1  : ( 7.3337604116209745 ,  9.93199698027032 )

Q 2  : ( 11.0 ,  5.0 )
takeoff pt 2  : ( 10.277139562802187 ,  13.918909028569773 )
landing pt 2  : ( 13.770392330412928 ,  18.649341089759655 )

Q 3  : ( 28.0 ,  41.0 )
takeoff pt 3  : ( 16.71338288186218 ,  22.63521853190723 )
landing pt 3  : ( 21.236337104945452 ,  28.237261387238597 )

Q 4  : ( 34.0 ,  39.0 )
takeoff pt 4  : ( 21.236337113793244 ,  28.237261394696677 )
landing pt 4  : ( 19.011916103666515 ,  26.833985607177826 )

Q 5  : ( 25.0 ,  40.0 )
takeoff pt 5  : ( 19.011916094680597 ,  26.833985599887303 )
landing pt 5  : ( 14.612258074908608 ,  21.134598979002256 )



In [12]:
m.getVars()

[<gurobi.Var Qx[1] (value 11.0)>,
 <gurobi.Var Qy[1] (value 0.0)>,
 <gurobi.Var pto_x[1] (value 4.231666144280172)>,
 <gurobi.Var pto_y[1] (value 5.731214118730556)>,
 <gurobi.Var pl_x[1] (value 7.3337604116209745)>,
 <gurobi.Var pl_y[1] (value 9.93199698027032)>,
 <gurobi.Var t1[1] (value 0.1167182997231709)>,
 <gurobi.Var t2[1] (value 0.13244389936436518)>,
 <gurobi.Var t[1] (value 0.29011261183689074)>,
 <gurobi.Var T[1] (value 0.3957872825272266)>,
 <gurobi.Var w[1,1] (value 0.0)>,
 <gurobi.Var w[1,2] (value 0.0)>,
 <gurobi.Var w[1,3] (value -0.0)>,
 <gurobi.Var w[1,4] (value -0.0)>,
 <gurobi.Var w[1,5] (value 1.0)>,
 <gurobi.Var Qx[2] (value 11.0)>,
 <gurobi.Var Qy[2] (value 5.0)>,
 <gurobi.Var pto_x[2] (value 10.277139562802187)>,
 <gurobi.Var pto_y[2] (value 13.918909028569773)>,
 <gurobi.Var pl_x[2] (value 13.770392330412928)>,
 <gurobi.Var pl_y[2] (value 18.649341089759655)>,
 <gurobi.Var t1[2] (value 0.11880214160236731)>,
 <gurobi.Var t2[2] (value 0.16764115597397966)>,
 <gu

In [13]:
w

{(1, 1): <gurobi.Var w[1,1] (value 0.0)>,
 (1, 2): <gurobi.Var w[1,2] (value 0.0)>,
 (1, 3): <gurobi.Var w[1,3] (value -0.0)>,
 (1, 4): <gurobi.Var w[1,4] (value -0.0)>,
 (1, 5): <gurobi.Var w[1,5] (value 1.0)>,
 (2, 1): <gurobi.Var w[2,1] (value -0.0)>,
 (2, 2): <gurobi.Var w[2,2] (value -0.0)>,
 (2, 3): <gurobi.Var w[2,3] (value -0.0)>,
 (2, 4): <gurobi.Var w[2,4] (value 1.0)>,
 (2, 5): <gurobi.Var w[2,5] (value -0.0)>,
 (3, 1): <gurobi.Var w[3,1] (value -0.0)>,
 (3, 2): <gurobi.Var w[3,2] (value 1.0)>,
 (3, 3): <gurobi.Var w[3,3] (value -0.0)>,
 (3, 4): <gurobi.Var w[3,4] (value 0.0)>,
 (3, 5): <gurobi.Var w[3,5] (value -0.0)>,
 (4, 1): <gurobi.Var w[4,1] (value -0.0)>,
 (4, 2): <gurobi.Var w[4,2] (value -0.0)>,
 (4, 3): <gurobi.Var w[4,3] (value 1.0)>,
 (4, 4): <gurobi.Var w[4,4] (value -0.0)>,
 (4, 5): <gurobi.Var w[4,5] (value 0.0)>,
 (5, 1): <gurobi.Var w[5,1] (value 1.0)>,
 (5, 2): <gurobi.Var w[5,2] (value -0.0)>,
 (5, 3): <gurobi.Var w[5,3] (value 0.0)>,
 (5, 4): <gurobi.Var 

In [14]:
Qx

{1: <gurobi.Var Qx[1] (value 11.0)>,
 2: <gurobi.Var Qx[2] (value 11.0)>,
 3: <gurobi.Var Qx[3] (value 28.0)>,
 4: <gurobi.Var Qx[4] (value 34.0)>,
 5: <gurobi.Var Qx[5] (value 25.0)>}